In [ ]:
def daily_gt(keyword, start, end, hl='en-US', tz=360):
  import pandas as pd 
  import numpy as np # numpy y pandas to data wrangling 
  from datetime import datetime, timedelta # to work w date
  from pytrends.request import TrendReq
  # función para dividir rango de fechas en segmentos 
  def date_range(start, end, intv):
      start = datetime.strptime(start,"%Y-%m-%d")
      end = datetime.strptime(end,"%Y-%m-%d")
      diff = (end  - start ) / intv
      for i in range(intv):
          yield (start + diff * i).strftime("%Y-%m-%d")
      yield end.strftime("%Y-%m-%d")


  pytrends = TrendReq(hl=hl, tz=tz) 

  # generación de lista de fechas a utilizar
  firstDate = datetime.strptime(start,"%Y-%m-%d")
  lastDate = datetime.strptime(end,"%Y-%m-%d")
  aux = (lastDate - firstDate)/90
  intv = aux.days
  timelist = list(date_range(start, end, intv))
  
  # armamos lista vacía para guardar los resultados
  var_dict={}
  for x in range(0, len(keyword)):
    varName = keyword[x]
    print(f'{x}: {varName}')
    dataset = pd.DataFrame(columns = [varName])

    for i in range(0, len(timelist)-1):
      if timelist[i] != start:
        startAux = datetime.strptime(timelist[i], "%Y-%m-%d") + timedelta(days=1)
        startNew = startAux.strftime("%Y-%m-%d")
      else:
        startAux = datetime.strptime(timelist[i], "%Y-%m-%d")
        startNew = startAux.strftime("%Y-%m-%d")
      print(f'Iteration from {startNew} to {timelist[i+1]}\n')
      pytrends.build_payload(kw_list=[varName], cat=0, timeframe=f'{startNew} {timelist[i+1]}') 
      data = pytrends.interest_over_time()
      if not data.empty:
        data = pd.DataFrame(data.drop(labels=['isPartial'],axis='columns'))
        data['year'] = data.index.year
        data['month'] = data.index.month 
        data['day'] = data.index.day 
        dataset = dataset.append(data)
    pytrends.build_payload(kw_list=[varName], cat=0, timeframe='all') 
    historical_data = pytrends.interest_over_time()
    historical_data = pd.DataFrame(historical_data.drop(labels=['isPartial'], axis='columns'))
    historical_data['year'] = historical_data.index.year
    historical_data['month'] = historical_data.index.month 
    historical_data.rename(columns = {f'{varName}':f'{varName}_historical'}, inplace = True)
    dataset = result = pd.merge(dataset, historical_data, on=["year", "month"])
    dataset[f'{varName}_adjusted'] = dataset[f'{varName}']*(dataset[f'{varName}_historical']/100) 
    var_dict[varName] = pd.DataFrame(dataset)